In [1]:
import glob
import pandas as pd
import re

## Attempting to load in and concat all of the spreadsheets

In [2]:
def loadData(path="data/raw/"):
    print('Loading data...')
    all_files = glob.glob(path + "*.ods")
    all_csvs = [pd.read_excel(filename, header=3, engine="odf") for filename in all_files]
    return pd.concat(all_csvs, axis=0, ignore_index=True)

In [3]:
df = pd.read_excel('data/raw/prison-pop-august-2023.ods', header=3, engine="odf")
df.head()

,Prison Name,Baseline CNA,In Use CNA,Operational Capacity,Population *
0,Altcourse,780.0,780.0,1164,1149.0
1,Ashfield,416.0,416.0,412,411.0
2,Askham Grange,128.0,128.0,128,94.0
3,Aylesbury,402.0,402.0,402,399.0
4,Bedford,268.0,229.0,400,358.0


In [5]:
df_concat = loadData()

Loading data...


In [6]:
df_concat

,Prison Name,Baseline CNA,In Use CNA,Operational Capacity,Population *
0,Altcourse,780.0,780.0,1164,1149.0
1,Ashfield,416.0,416.0,412,411.0
2,Askham Grange,128.0,128.0,128,94.0
3,Aylesbury,402.0,402.0,402,399.0
4,Bedford,268.0,229.0,400,358.0
...,...,...,...,...,...
1239,Her Majesty's Prison & Probation Service (HMPP...,NaN,NaN,NaN,NaN
1240,This is published in the HMPPS Annual Digest ...,NaN,NaN,NaN,NaN
1241,NaN,NaN,NaN,NaN,NaN
1242,Establishments Exceeding their Operational Cap...,NaN,NaN,NaN,NaN


This has worked but there is no identifying date column. So I need to work through and see if I can extract and infer the date from the filename

In [17]:
all_files = glob.glob("data/raw/" + "*.ods")
all_csvs = [pd.read_excel(filename, header=3, engine="odf") for filename in all_files]

In [18]:
all_files

['data/raw/prison-pop-august-2023.ods',
 'data/raw/prison-pop-february-2023.ods',
 'data/raw/prison-pop-june-2023.ods',
 'data/raw/prison-pop-july-2023.ods',
 'data/raw/prison-pop-april-2023.ods',
 'data/raw/prison-pop-january-2023.ods',
 'data/raw/prison-pop-may-2023.ods',
 'data/raw/prison-population-march-2023.ods']

In [19]:
date_text = re.findall(r'(?:data/raw/prison-pop-)(.*)(?:.ods)', all_files[0])[0]

In [20]:
date_text

'august-2023'

In [ ]:
pd.to_datetime(date_text, format='%B-%Y')

Timestamp('2023-08-01 00:00:00')

In [59]:
all_csvs[0]['date'] = pd.to_datetime(date_text, format='%B-%Y')
all_csvs[0]

,Prison Name,Baseline CNA,In Use CNA,Operational Capacity,Population *,date
0,Altcourse,780.0,780.0,1164,1149.0,2023-08-01
1,Ashfield,416.0,416.0,412,411.0,2023-08-01
2,Askham Grange,128.0,128.0,128,94.0,2023-08-01
3,Aylesbury,402.0,402.0,402,399.0,2023-08-01
4,Bedford,268.0,229.0,400,358.0,2023-08-01
...,...,...,...,...,...,...
151,Her Majesty's Prison & Probation Service (HMPP...,NaN,NaN,NaN,NaN,2023-08-01
152,This is published in the HMPPS Annual Digest ...,NaN,NaN,NaN,NaN,2023-08-01
153,NaN,NaN,NaN,NaN,NaN,2023-08-01
154,Establishments Exceeding their Operational Cap...,NaN,NaN,NaN,NaN,2023-08-01


Great, this logic works, so let's consolidate this into a loop within the `loadData()` function

In [10]:
def loadData(path="data/raw/"):
    print('Loading data...')
    all_files = glob.glob(path + "*.ods")
    all_odfs = [pd.read_excel(filename, header=3, engine="odf") for filename in all_files]
    all_dates = [re.findall(r'(?:data/raw/prison-pop-)(.*)(?:.ods)', all_files[i])[0] for i in range(len(all_files)-1)]

    
    
    return pd.concat(all_odfs, axis=0, ignore_index=True)

In [11]:
df = loadData()

Loading data...


In [15]:
len(df)

8

In [13]:
df[0]

,Prison Name,Baseline CNA,In Use CNA,Operational Capacity,Population *
0,Altcourse,780.0,780.0,1164,1149.0
1,Ashfield,416.0,416.0,412,411.0
2,Askham Grange,128.0,128.0,128,94.0
3,Aylesbury,402.0,402.0,402,399.0
4,Bedford,268.0,229.0,400,358.0
...,...,...,...,...,...
151,Her Majesty's Prison & Probation Service (HMPP...,NaN,NaN,NaN,NaN
152,This is published in the HMPPS Annual Digest ...,NaN,NaN,NaN,NaN
153,NaN,NaN,NaN,NaN,NaN
154,Establishments Exceeding their Operational Cap...,NaN,NaN,NaN,NaN


In [66]:
all_files = glob.glob("data/raw/" + "*.ods")
all_odfs = [pd.read_excel(filename, header=3, engine="odf") for filename in all_files]
all_dates = [re.findall(r'(?:data/raw/prison-pop-)(.*)(?:.ods)', all_files[i])[0] for i in range(len(all_files)-1)]



In [67]:
all_dates

['august-2023',
 'february-2023',
 'june-2023',
 'july-2023',
 'april-2023',
 'january-2023',
 'may-2023']

In [27]:
all_files

['data/raw/prison-pop-august-2023.ods',
 'data/raw/prison-pop-february-2023.ods',
 'data/raw/prison-pop-june-2023.ods',
 'data/raw/prison-pop-july-2023.ods',
 'data/raw/prison-pop-april-2023.ods',
 'data/raw/prison-pop-january-2023.ods',
 'data/raw/prison-pop-may-2023.ods',
 'data/raw/prison-population-march-2023.ods']

In [63]:
date_text = re.findall(r'(?:data/raw/prison-pop-)(.*)(?:.ods)', all_files[-1])
date_text

[]